In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv(r"C:\Users\hp\Downloads\Anomaly-Detection-SocialMedia\test\data\bot_detection_data.csv")
df.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [3]:
df.columns

Index(['User ID', 'Username', 'Tweet', 'Retweet Count', 'Mention Count',
       'Follower Count', 'Verified', 'Bot Label', 'Location', 'Created At',
       'Hashtags'],
      dtype='object')

In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

In [5]:
# splitting 
X = df['Tweet']
y = df['Bot Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# tokenization using BERTweet tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")


c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [7]:
# test tokenization with a sample text

sample_text = "Twitter bot detection is interesting!"
tokens = tokenizer(sample_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
tokens

{'input_ids': tensor([[    0,   562,  5178, 33896,    17, 48012,   856,    12,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

# 1. Create Dataset Class:

In [8]:
from torch.utils.data import Dataset

class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

# Convert the labels to match the format
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Create the datasets
train_dataset = TweetDataset(train_encodings, y_train)
test_dataset = TweetDataset(test_encodings, y_test)


# 2. Load the Pre-trained Model:

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)


c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 3. Define Training Arguments:

In [12]:
from transformers import TrainingArguments

# Example training arguments (customize as needed)
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save model checkpoints and outputs
    evaluation_strategy="steps",  # Perform evaluation at regular intervals
    save_steps=500,  # Save model every 500 steps
    save_total_limit=2,  # Keep only the last 2 checkpoints
    logging_dir="./logs",  # Directory for logs
    logging_steps=100,  # Log every 100 steps
    per_device_train_batch_size=8,  # Adjust based on your GPU memory
    per_device_eval_batch_size=8,  # Adjust based on your GPU memory
    num_train_epochs=3,  # Total number of epochs
    load_best_model_at_end=True,  # Automatically load the best model
)

# 4. Initialize the Trainer:

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

TypeError: Accelerator.__init__() got an unexpected keyword argument 'dispatch_batches'

# 5. Train the model:

In [ ]:
trainer.train()

# 6. Evaluation:

In [ ]:
results = trainer.evaluate()
results